In [5]:
using Laplacians

In [6]:
include("../src/newFastSampler2.jl")

epsequal (generic function with 1 method)

## bounds error

In [ ]:
p = reshape(readdlm("boundsErr.txt"),96)

In [ ]:
newSampler(p)

In [ ]:
Tv = Float64

In [ ]:
    n = length(p)

    # generate the Sampler
    F = Array{Tv}(n)
    A = Array{Int64}(n)
    V = Array{Int64}(n)
    
    # generate a normalized p, so that it sums up to n
    newp = n * copy(p) / sum(p)

    # fill up the arays A and V
    pos = 0                        # number of filled or half filled buckets
    posFilled = 0                # number of filled buckets

    for i in 1:n
        if newp[i] < 1 #&& !epsequal(newp[i], 1)
            pos = pos + 1
            A[pos] = i
            F[pos] = newp[i]
        end
    end

    #residualError = 0

    for i in 1:n
        if newp[i] >= 1 #|| epsequal(newp[i], 1)
            val = newp[i]
            assert(!(pos == n && i < n))
            while val >= 1 #|| epsequal(val, 1)
                if posFilled < pos
                    # fill up a bucket already containing a small element, and subtract that value from the current element
                    posFilled = posFilled + 1
                    V[posFilled] = i
                    val = val - (1 - F[posFilled])
                else
                    # if pos == n
                    #     writedlm("boundsErr.txt",p)
                    # end
                    
                    # create a bucket with only one large element. note posFilled = pos
                    pos = pos + 1
                    posFilled = posFilled + 1
                    A[pos] = V[pos] = i
                    F[pos] = 1
                    val = val - 1
                end
            end

            # treat the case in which val becomes smaller than 1. Have to do the pos < n check because of precision errors
            if val > 0 && pos < n #&& !epsequal(val, 0) && pos < n
                pos = pos + 1
                A[pos] = i
                F[pos] = val
            end

            # residualError += val
        end
    end

err = 0
 while posFilled < n
     # if we are in this case, posFilled should be equal to pos - 1
     posFilled = posFilled + 1
     V[posFilled] = A[posFilled]
    err += 1-F[posFilled]
 end 

println("err = ",err)

In [ ]:
(pos,posFilled)

In [10]:
srand(0)

MersenneTwister(Base.dSFMT.DSFMT_state(Int32[748398797,1073523691,-1738140313,1073664641,-1492392947,1073490074,-1625281839,1073254801,1875112882,1073717145  …  943540191,1073626624,1091647724,1073372234,-1273625233,-823628301,835224507,991807863,382,-842150451]),[1.29034,1.69045,1.17545,1.75888,1.62698,1.50235,1.0131,1.68214,1.94385,1.13279  …  1.22344,1.11442,1.5363,1.49708,1.59543,1.29545,1.74436,1.17272,1.46399,1.15197],382,UInt32[0x00000000])

In [29]:
m = 0
@time for i = 1:10000
    p = rand(1:100,100000)
    m = max(newSampler(p*1.0),m)
end
m

 76.396014 seconds (199.87 k allocations: 59.610 GB, 8.00% gc time)


5.49582601649945e-12

In [20]:
rand(1:80,1000)

1000-element Array{Int64,1}:
 72
 16
 71
 45
 24
  7
 26
 21
 80
  7
 56
 35
 57
  ⋮
 57
 38
  2
 67
 30
 15
 17
 71
 31
 42
 20
 80